### Notebook to demonstrate AutoML workflow for TAO classification models

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://developer.nvidia.com/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)


### Learning Objective

This AutoML notebook applies to identifying the optimal hyperparameters (e.g., learning rate, batch size, weight regularizer, number of layers, etc.) in order to obtain better accuracy results or converge faster on AI models for classification application. 
- Take a pretrained model and choose automl algorithm/parameters to start AutoML train.
- At the end of an AutoML run, you will receive a config file that specifies the best performing model, along with the binary model file to deploy it to your application.


### AutoML Workflow

User starts with selecting model topology, create and upload dataset, configuring parameters, training with AutoML to comparing the model.

![image](https://raw.githubusercontent.com/vpraveen-nv/model_card_images/main/api/automl_workflow.png)


### Table of contents

1. [Create and upload datasets](#head-1)
1. [List the created datasets](#head-2)
1. [Dataset convert Action](#head-3)
1. [Create model](#head-4)
1. [List models](#head-5)
1. [Assign train, eval datasets](#head-6)
1. [Assign PTM](#head-7)
1. [Set AutoML related configurations](#head-8)
1. [Actions](#head-9)
1. [AutoML Train](#head-10)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

In [ ]:
import json
import os
import requests
import uuid
import time
from IPython.display import clear_output

### FIXME

1. Assign a model_name in FIXME 1
2. Assign a workdir in FIXME 2
3. Assign the ip_address and port_number in FIXME 3 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
4. Assign the ngc_api_key variable in FIXME 4
5. Choose between default and custom dataset in FIXME5
6. Assign path of data directory in FIXME 6
7. Choose between Bayesian and Hyperband automl_algorithm in FIXME 7

In [ ]:
# Define model_name workspaces and other variables
# Available models (#FIXME 1):
# 1. classification - https://docs.nvidia.com/tao/tao-toolkit/text/image_classification.html
# 2. multitask_classification - https://docs.nvidia.com/tao/tao-toolkit/text/multitask_image_classification.html
# classification is the same as multi-class classification

model_name = "multitask_classification" # FIXME1 (Add the model name from the above mentioned list)
workdir = "workdir_classification" # FIXME2
host_url = "http://<ip_address>:<port_number>" # FIXME3 example: https://10.137.149.22:32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key = "<ngc_api_key>" # FIXME4 example: zZYtczM5amdtdDcwNjk0cnA2bGU2bXQ3bnQ6NmQ4NjNhMDItMTdmZS00Y2QxLWI2ZjktNmE5M2YxZTc0OGyM
dataset_to_be_used = "default" #FIXME5 example: default/custom; default for the dataset used in this tutorial notebook; custom for a different dataset

In [ ]:
# Exchange NGC_API_KEY for JWT
response = requests.get(f"{host_url}/api/v1/login/{ngc_api_key}")
user_id = response.json()["user_id"]
print("User ID",user_id)
token = response.json()["token"]
print("JWT",token)

# Set base URL
base_url = f"{host_url}/api/v1/user/{user_id}"
print("API Calls will be forwarded to",base_url)

headers = {"Authorization": f"Bearer {token}"}

In [ ]:
# Creating workdir
if not os.path.isdir(workdir):
    os.makedirs(workdir)

### Create datasets <a class="anchor" id="head-1"></a>

**For multi-class classification:**

We will be using the pascal `VOC dataset` for the tutorial. To find more details please visit [here](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html#devkit). Please download the [dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar) to the environment variable `$DATA_DIR`.

**If using custom dataset; it should follow this dataset structure, and skip running** "**Split dataset into train and val sets**"
```
DATA_DIR
├── images_test
│   ├── class_name_1
│   │   ├── image_name_1.jpg
│   │   ├── image_name_2.jpg
│   │   ├── ...
|   |   ... 
│   └── class_name_n
│       ├── image_name_3.jpg
│       ├── image_name_4.jpg
│       ├── ...
├── images_train
│   ├── class_name_1
│   │   ├── image_name_5.jpg
│   │   ├── image_name_6.jpg
|   |   ...
│   └── class_name_n
│       ├── image_name_7.jpg
│       ├── image_name_8.jpg
│       ├── ...
|
└── images_val
    ├── class_name_1
    │   ├── image_name_9.jpg
    │   ├── image_name_10.jpg
    │   ├── ...
    |   ...
    └── class_name_n
        ├── image_name_11.jpg
        ├── image_name_12.jpg
        ├── ...
```
- Each class name folder should contain the images corresponding to that class
- Same class name folders should be present across images_test, images_train and images_val

**For multi-task classification:**

We will be using the `Fashion Product Images (Small)` for the tutorial. This dataset is available on Kaggle.In this tutorial, our trained classification network will perform three tasks: article category classification, base color classification and target season classification.

To download the dataset, you will need a Kaggle account. After login, you can download the dataset zip file [here](https://www.kaggle.com/paramaggarwal/fashion-product-images-small). The downloaded file is archive.zip with a subfolder called myntradataset. Unzip contents in this subfolder to your workdir created in the cell above and you should have a folder called images and a CSV file called styles.csv

**If using custom dataset; it should follow this dataset structure**
```
DATA_DIR
├── images
│   ├── image_name_1.jpg
│   ├── image_name_2.jpg
|   |   ├── ...
├── styles.csv
```

In [ ]:
DATA_DIR = model_name # FIXME6
os.environ['DATA_DIR']= DATA_DIR
!mkdir -p $DATA_DIR

In [ ]:
if dataset_to_be_used == "default":
    if model_name == "classification":
        if not os.path.exists(os.path.join(DATA_DIR,"VOCtrainval_11-May-2012.tar")):
            print("Download VOC tar data into ", DATA_DIR)
        else:
            !tar -xf $DATA_DIR/VOCtrainval_11-May-2012.tar -C $DATA_DIR
    elif model_name == "multitask_classification":
        if not os.path.exists(os.path.join(DATA_DIR,"archive.zip")):
            print(f"Download Fashion zip data into ", DATA_DIR)
        else:
            !unzip -uq $DATA_DIR/archive.zip -d $DATA_DIR/

### Split dataset into train and val sets

In [ ]:
# Check the dataset is present
if model_name == "classification" and dataset_to_be_used == "default":
    !if [ ! -d $DATA_DIR/VOCdevkit ]; then echo 'Images folder NOT found.'; else echo 'Found images folder.';fi
    !rm -rf $DATA_DIR/split
elif model_name == "multitask_classification":
    !if [ ! -d $DATA_DIR/images ]; then echo 'Images folder NOT found.'; else echo 'Found images folder.';fi
    !if [ ! -f $DATA_DIR/styles.csv ]; then echo 'CSV file NOT found.'; else echo 'Found CSV file.';fi
    # Create subdirectories and remove existing files in them
    !mkdir -p $DATA_DIR/images_train && rm -rf $DATA_DIR/images_train/*
    !mkdir -p $DATA_DIR/images_val && rm -rf $DATA_DIR/images_val/*
    !mkdir -p $DATA_DIR/images_test && rm -rf $DATA_DIR/images_test/*

In [ ]:
# Split dataset into train and val sets
!python3 -m pip install numpy
!python3 -m pip install pandas
if model_name == "classification" and dataset_to_be_used == "default":
    !python3 -m pip install tqdm
    from os.path import join as join_path
    import os
    import glob
    import re
    import shutil

    DATA_DIR=os.environ.get('DATA_DIR')
    source_dir = join_path(DATA_DIR, "VOCdevkit/VOC2012")
    target_dir = join_path(DATA_DIR, "formatted")

    suffix = '_trainval.txt'
    classes_dir = join_path(source_dir, "ImageSets", "Main")
    images_dir = join_path(source_dir, "JPEGImages")
    classes_files = glob.glob(classes_dir+"/*"+suffix)
    for file in classes_files:
        # get the filename and make output class folder
        classname = os.path.basename(file)
        if classname.endswith(suffix):
            classname = classname[:-len(suffix)]
            target_dir_path = join_path(target_dir, classname)
            if not os.path.exists(target_dir_path):
                os.makedirs(target_dir_path)
        else:
            continue
        print(classname)

        with open(file) as f:
            content = f.readlines()

        for line in content:
            tokens = re.split('\s+', line)
            if tokens[1] == '1':
                # copy this image into target dir_path
                target_file_path = join_path(target_dir_path, tokens[0] + '.jpg')
                src_file_path = join_path(images_dir, tokens[0] + '.jpg')
                shutil.copyfile(src_file_path, target_file_path)
    
    from random import shuffle
    from tqdm import tqdm

    DATA_DIR=os.environ.get('DATA_DIR')
    SOURCE_DIR=os.path.join(DATA_DIR, 'formatted')
    TARGET_DIR=os.path.join(DATA_DIR,'split')
    # list dir
    print(os.walk(SOURCE_DIR))
    dir_list = next(os.walk(SOURCE_DIR))[1]
    # for each dir, create a new dir in split
    for dir_i in tqdm(dir_list):
        newdir_train = os.path.join(TARGET_DIR, 'images_train', dir_i)
        newdir_val = os.path.join(TARGET_DIR, 'images_val', dir_i)
        newdir_test = os.path.join(TARGET_DIR, 'images_test', dir_i)

        if not os.path.exists(newdir_train):
                os.makedirs(newdir_train)
        if not os.path.exists(newdir_val):
                os.makedirs(newdir_val)
        if not os.path.exists(newdir_test):
                os.makedirs(newdir_test)

        img_list = glob.glob(os.path.join(SOURCE_DIR, dir_i, '*.jpg'))
        # shuffle data
        shuffle(img_list)

        for j in range(int(len(img_list)*0.7)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'images_train', dir_i))

        for j in range(int(len(img_list)*0.7), int(len(img_list)*0.8)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'images_val', dir_i))

        for j in range(int(len(img_list)*0.8), len(img_list)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'images_test', dir_i))

    print('Done splitting dataset.')

elif model_name == "multitask_classification" and dataset_to_be_used == "default":
    !python3 -m pip install numpy
    !python3 -m pip install pandas
    import os
    import shutil
    import numpy as np
    import pandas as pd

    df = pd.read_csv(os.environ['DATA_DIR'] + '/styles.csv', error_bad_lines=False, warn_bad_lines=False)
    df = df[['id', 'baseColour', 'subCategory', 'season']]
    df = df.dropna()
    category_cls = df.subCategory.value_counts()[:10].index # 10-class multitask-classification
    season_cls = ['Spring', 'Summer', 'Fall', 'Winter'] # 4-class multitask-classification
    color_cls = df.baseColour.value_counts()[:11].index # 11-class multitask-classification

    # Get all valid rows
    df = df[df.subCategory.isin(category_cls) & df.season.isin(season_cls) & df.baseColour.isin(color_cls)]
    df.columns = ['fname', 'base_color', 'category', 'season']
    df.fname = df.fname.astype(str)
    df.fname = df.fname + '.jpg'

    # remove entries whose image file is missing
    all_img_files = os.listdir(os.environ['DATA_DIR'] + '/images')
    df = df[df.fname.isin(all_img_files)]

    idx = np.arange(len(df))
    np.random.shuffle(idx)

    train_split_idx = int(len(df)*0.8)
    train_df = df.iloc[idx[:train_split_idx]]
    val_df = df.iloc[idx[train_split_idx:]]

    # Add a simple sanity check
    assert len(train_df.season.unique()) == 4 and len(train_df.base_color.unique()) == 11 and \
        len(train_df.category.unique()) == 10, 'Training set misses some classes, re-run this cell!'
    assert len(val_df.season.unique()) == 4 and len(val_df.base_color.unique()) == 11 and \
        len(val_df.category.unique()) == 10, 'Validation set misses some classes, re-run this cell!'

    for image_name in train_df["fname"]:
        source_file_name = os.path.join(DATA_DIR, "images", image_name)
        destination_file_name = os.path.join(DATA_DIR, "images_train", image_name)
        shutil.copy(source_file_name, destination_file_name)

    for image_name in val_df["fname"]:
        source_file_name = os.path.join(DATA_DIR, "images", image_name)
        destination_file_name = os.path.join(DATA_DIR, "images_train", image_name)
        shutil.copy(source_file_name, destination_file_name)
        destination_file_name = os.path.join(DATA_DIR, "images_val", image_name)
        shutil.copy(source_file_name, destination_file_name)

    # save processed data labels
    train_df.to_csv(os.environ['DATA_DIR'] + '/train.csv', index=False)
    val_df.to_csv(os.environ['DATA_DIR'] + '/val.csv', index=False)

### Verify the dataset split

In [ ]:
# verify
if model_name == "classification":
    !if [ ! -d $DATA_DIR/split/images_train ]; then echo 'train folder NOT found.'; else echo 'Found train images folder.';fi
    !if [ ! -d $DATA_DIR/split/images_val ]; then echo 'val folder NOT found.'; else echo 'Found val images folder.';fi
    !if [ ! -d $DATA_DIR/split/images_test ]; then echo 'test folder NOT found.'; else echo 'Found test images folder.';fi
elif model_name == "multitask_classification":
    import pandas as pd

    print("Number of images in the train set. {}".format(
        len(pd.read_csv(os.environ['DATA_DIR'] + '/train.csv'))
    ))
    print("Number of images in the validation set. {}".format(
        len(pd.read_csv(os.environ['DATA_DIR'] + '/val.csv'))
    ))

### Create Tar files to upload

In [ ]:
if model_name == "classification":
    !tar -C $DATA_DIR/split/ -czf classification_train.tar.gz images_train
    !tar -C $DATA_DIR/split/ -czf classification_val.tar.gz images_val
elif model_name == "multitask_classification":
    !tar -C $DATA_DIR/ -czf mt_classification_train.tar.gz images_train train.csv val.csv
    !tar -C $DATA_DIR/ -czf mt_classification_val.tar.gz images_val val.csv

In [ ]:
if model_name == "classification":
    train_dataset_path =  "classification_train.tar.gz"
    eval_dataset_path = "classification_val.tar.gz"
elif model_name == "multitask_classification":
    train_dataset_path =  "mt_classification_train.tar.gz"
    eval_dataset_path = "mt_classification_val.tar.gz"

### Create and upload datasets

In [ ]:
# Create train dataset
ds_type = "image_classification"
if model_name == "classification":
    ds_format = "default"
elif model_name == "multitask_classification":
    ds_format = "custom"

data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Train dataset",
                       "description":"My train dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(train_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

In [ ]:
# Create eval dataset
ds_type = "image_classification"
ds_format = "default"
data = json.dumps({"type":ds_type,"format":ds_format})

endpoint = f"{base_url}/dataset"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
eval_dataset_id = response.json()["id"]

In [ ]:
# Update
dataset_information = {"name":"Eval dataset",
                       "description":"My eval dataset"}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/dataset/{eval_dataset_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

In [ ]:
# Upload
files = [("file",open(eval_dataset_path,"rb"))]

endpoint = f"{base_url}/dataset/{eval_dataset_id}/upload"

response = requests.post(endpoint, files=files, headers=headers)

print(response)
print(response.json())

### List the created datasets <a class="anchor" id="head-2"></a>

In [ ]:
endpoint = f"{base_url}/dataset"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("id\t\t\t\t\t type\t\t\t format\t\t name")
for rsp in response.json():
    print(rsp["id"],"\t",rsp["type"],"\t",rsp["format"],"\t\t",rsp["name"])

### Create model <a class="anchor" id="head-4"></a>

In [ ]:
network_arch = model_name
if network_arch == "classification":
    encode_key = "nvidia_tlt"
else:
    encode_key = "tlt_encode"
data = json.dumps({"network_arch":network_arch,"encryption_key":encode_key})

endpoint = f"{base_url}/model"

response = requests.post(endpoint,data=data,headers=headers)

print(response)
print(response.json())
model_id = response.json()["id"]

### List models <a class="anchor" id="head-5"></a>

In [ ]:
endpoint = f"{base_url}/model"

response = requests.get(endpoint, headers=headers)

print(response)
# print(response.json()) ## Uncomment for verbose list output
print("model id\t\t\t     network architecture")
for rsp in response.json():
    print(rsp["id"],rsp["network_arch"])

### Assign train, eval datasets <a class="anchor" id="head-6"></a>

- Note: make sure the order for train_datasets is [source ID, target ID]
- eval_dataset is kept same as target for demo purposes
- inference_dataset is kept as target for chaining with hifigan finetune

In [ ]:
dataset_information = {"train_datasets":[dataset_id],
                       "eval_dataset":eval_dataset_id}
data = json.dumps(dataset_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### Assign PTM <a class="anchor" id="head-7"></a>

Search for pretrained models on NGC and assign it to the model

In [ ]:
# Assigning pretrained models to different classification models
# print base_url+"/model" to get the details of all pretrained models and make the appropriate changes to this map for experiments like for example 
# you are changing the number of layers to 34, then you have to make the appropriate change in the pretrained model name
# print(base_url+"/model")
pretrained_map = {"classification" : "pretrained_classification:resnet18",
                  "multitask_classification" : "pretrained_classification:resnet10"}

In [ ]:
# Get pretrained model for classification
model_list = f"{base_url}/model"
response = requests.get(model_list, headers=headers)

response_json = response.json()

# Search for ptm with given ngc path
ptm_id = None
for rsp in response_json:
    if rsp["network_arch"] == network_arch and rsp["ngc_path"].endswith(pretrained_map[network_arch]):
        ptm_id = rsp["id"]
        print("Metadata for model with requested NGC Path")
        print(rsp)
        break
mt_class_ptm = ptm_id

In [ ]:
ptm_information = {"ptm":mt_class_ptm}
data = json.dumps(ptm_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
print(response.json())

### View hyperparameters that are enabled for AutoML by default

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/train/schema"

response = requests.get(endpoint, headers=headers)
specs = response.json()["automl_default_parameters"]

import json
print(json.dumps(specs, sort_keys=True, indent=4))

### Set AutoML related configurations <a class="anchor" id="head-8"></a>
Refer to these hyper-links to see the parameters supported by each network and add more parameters if necessary in addition to the default automl enabled parameters: [Multiclass_classification](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_action_specs.html#train), 
[Multitask_classification](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_action_specs.html#id27)

In [ ]:
# Choose automl algorithm between "Bayesian" and "HyperBand".
automl_algorithm="Bayesian" # FIXME7 example: Bayesian/HyperBand
metric="kpi" # don't change, more metrics will be supported in the future

additional_automl_parameters = [] #Refer to parameter list mentioned in the above links and add any extra parameter in addition to the default enabled ones
remove_default_automl_parameters = [] #Remove any hyperparameters that are enabled by default for AutoML

automl_information = {"automl_enabled":True,
                      "automl_algorithm":automl_algorithm,
                      "metric":metric,
                      "automl_add_hyperparameters":str(additional_automl_parameters),
                      "automl_remove_hyperparameters":str(remove_default_automl_parameters)
                     }
data = json.dumps(automl_information)

endpoint = f"{base_url}/model/{model_id}"

response = requests.patch(endpoint, data=data, headers=headers)

print(response)
import json
print(json.dumps(response.json(), sort_keys=True, indent=4))

### Actions <a class="anchor" id="head-9"></a>

For all actions:
1. Get default spec schema and derive the default values
2. Modify defaults if needed
3. Post spec dictionary to the service
4. Run model action
5. Monitor job using retrieve
6. Download results using job download endpoint (if needed)

In [ ]:
job_map = {}

### AutoML Train <a class="anchor" id="head-10"></a>

In [ ]:
# Get default spec schema
endpoint = f"{base_url}/model/{model_id}/specs/train/schema"

response = requests.get(endpoint, headers=headers)

specs = response.json()["default"]

import json
print(json.dumps(specs, sort_keys=True, indent=4))

In [ ]:
# Override any of the parameters listed in the previous cell as required
# Example for multitask-classification (for each network the parameter key might be different)
specs["training_config"]["num_epochs"] = 10
# Example for classification
# specs["train_config"]["n_epochs"] = 80

In [ ]:
# Post spec
data = json.dumps(specs)

endpoint = f"{base_url}/model/{model_id}/specs/train"

response = requests.post(endpoint,data=data, headers=headers)

print(response)
import json
print(json.dumps(response.json(), sort_keys=True, indent=4))

In [ ]:
# Run action
parent = None
actions = ["train"]
data = json.dumps({"job":parent,"actions":actions})

endpoint = f"{base_url}/model/{model_id}/job"

response = requests.post(endpoint, data=data, headers=headers)

print(response)
print(response.json())

job_map["train"] = response.json()[0]
print(job_map)

In [ ]:
# Monitor automl job status by repeatedly running this cell
# Training times for different models benchmarked on 1 GPU V100 machine can be found here: https://docs.nvidia.com/tao/tao-toolkit/text/automl/automl.html#results-of-automl-experiments

job_id = job_map['train']
endpoint = f"{base_url}/model/{model_id}/job/{job_id}"

while True:
    clear_output(wait=True)
    response = requests.get(endpoint, headers=headers)
    print(response)
    print(json.dumps(response.json(), sort_keys=True, indent=4))
    if response.json().get("status") in ["Done","Error"] or response.status_code not in (200,201):
        break
    time.sleep(15)

In [ ]:
## To Stop an AutoML JOB
#    1. Stop the 'Monitor automl job status by repeatedly running this cell' cell (the cell right before this cell) manually
#    2. Uncomment the snippet in the next cell and run the cell

In [ ]:
# job_id = job_map['train']
# endpoint = f"{base_url}/model/{model_id}/job/{job_id}/cancel"

# response = requests.post(endpoint, headers=headers)

# print(response)
# print(response.json())

In [ ]:
## Resume AutoML

In [ ]:
# Uncomment the below snippet if you want to resume an already stopped AutoML job and then run the 'Monitor automl job status by repeatedly running this cell' cell above (4th cell above from this cell)
# job_id = job_map['train']
# endpoint = f"{base_url}/model/{model_id}/job/{job_id}/resume"

# response = requests.post(endpoint, headers=headers)

# print(response)
# print(response.json())

In [ ]:
# Download automl job contents once the above job shows "Done" status
# Download output of automl (multitask_classification) train (Note: will take time)
job_id = job_map["train"]
endpoint = f'{base_url}/model/{model_id}/job/{job_id}/download'

# Save
temptar = f'{job_id}.tar.gz'
with requests.get(endpoint, headers=headers, stream=True) as r:
    r.raise_for_status()
    with open(temptar, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Untarring")

# Untar to destination
tar_command = f'tar -xvf {temptar} -C {workdir}/'
os.system(tar_command)
os.remove(temptar)
print(f"Results at {workdir}/{job_id}")
model_downloaded_path = f"{workdir}/{job_id}"

In [ ]:
# View best performing model's config, model file; Also view the results of all automl experiments
!python3 -m pip install pandas
import pandas as pd

best_model_path = f"{model_downloaded_path}/best_model"

if os.path.exists(best_model_path):        
    #List the binary model file
    print("\nCheckpoints for the best performing experiment")
    if os.path.exists(best_model_path+"/weights") and len(os.listdir(best_model_path+"/weights")) > 0:
        print(f"Folder: {best_model_path}/weights")
        print("Files:", os.listdir(best_model_path+"/weights"))
    else:
        print(f"Folder: {best_model_path}")
        print("Files:", os.listdir(best_model_path))

    experiment_artifacts = json.load(open(f"{best_model_path}/controller.json","r"))
    data_frame = pd.DataFrame(experiment_artifacts)
    # Print experiment id/number and the corresponding result
    print("\nResults of all experiments")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
        print(data_frame[["id","result"]])

    print("\nConfig/Spec file for the best performing experiment (recommendation_id.kitti with the maximum result value in the dataframe)")
    # List the recommendation config file of the best performing checkpoint(recommendation_id.kitti with the maximum result value in the dataframe)
    !ls {best_model_path}/*.kitti 